# Data Access with STAC

This notebook introduces [STAC](https://stacspec.org/), the SpatioTemproal Asset Catalog. The Planetary Computer hosts petabytes of data in Azure Blob Storage; STAC is what enables you to find the data you actually care about.

## STAC Concepts

We'll work our way up to searching millions of items for the few that cover an area or time of interest, but to use STAC effectively we need to briefly cover a few concepts. First, STAC is a *metadata* standard. It's all about organizing and exposing metadata like when a satellite scene was captured, what area of earth it covers, etc. Starting at the lowest level we have a STAC Item. 

In [ ]:
import pystac

item = pystac.read_file(
    "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-8-c2-l2/items/LC08_L2SP_023034_20210910_02_T1"
)
item

For remote-sensing data, an Item typically corresponds to a single scene. The core STAC specification requires things like the geometry:

In [ ]:
item.geometry

And the datetime

In [ ]:
item.datetime

STAC has many [extensions](https://stac-extensions.github.io/) for describing additional metadata. For example, the `eo` extension is a place to store information commonly available in electro-optical datasets. For example, the cloud-cover in an image:

In [ ]:
from pystac.extensions.eo import EOExtension

EOExtension.ext(item).cloud_cover

The [`proj`](https://github.com/stac-extensions/projection) extension defines metadata for geospatial projection information, like item's EPSG code:

In [ ]:
from pystac.extensions.projection import ProjectionExtension

ProjectionExtension.ext(item).epsg

Or the bounding box of the item in its native projection

In [ ]:
ProjectionExtension.ext(item).bbox

Or the geospatial transform, which might be familiar from GDAL:

In [ ]:
ProjectionExtension.ext(item.assets["SR_B2"]).transform

The last example showed accessing an **asset** from a STAC **item**. This is where the really interesting bits are. Recall that STAC is a metadata standard; you don't actually store the data "in STAC". Instead, you store a *link* to the data in an asset.

For remote-sensing datsets, it's common to have one asset per band, and perhaps some additional assets for links to thumbnails, documentation, or other related assets.

In [ ]:
item.assets

For example, we can use the `tilejson` asset, which describes how to render the item, to embed a preview of the item with ipyleaflet.

In [ ]:
from ipyleaflet import Map, TileLayer, GeoJSON, FullScreenControl
import shapely.geometry
import requests

center = shapely.geometry.shape(item.geometry).centroid.bounds[:2][::-1]

m = Map(center=center, zoom=12)
layer = TileLayer(
    url=requests.get(item.assets["tilejson"].href).json()["tiles"][0],
)
m.add_layer(layer)
m.add_control(FullScreenControl())

m.scroll_wheel_zoom = True
m

One thing to note about data from the Planetary Computer: The data files themselves (COGs in this case) are typically stored in *private* Blob Storage containers. If you just try to download the data, you'll get a 404 error.

In [ ]:
import requests

r = requests.get(item.assets["SR_B2"].href)
r.status_code

When you access data from the planetary computer, you need to *sign* it first. This requires another API call to an endpoint we run, which will give you a token to add to the end of the URLs. We provide a `planetary_computer` package to make this easy.

In [ ]:
import rasterio
import planetary_computer

signed_item = planetary_computer.sign(item)

ds = rasterio.open(signed_item.assets["SR_B2"].href)
ds.shape

You can sign assets anonymously, so you don't need an account with the Planetary Computer to work with its data.

STAC **items** are organized into **collections**. So a collection is just a grouping of items that all have some similar features (e.g. captured by the same sensor, share the same license, etc.). For example, Landsat 8 Collection-2 Level-2 is a collection. You can browse the collections hosted by the Planetary Computer at https://planetarycomputer.microsoft.com/catalog, for example [Landsat 8 Collection 2 Level-2](https://planetarycomputer.microsoft.com/dataset/landsat-8-c2-l2).

To recap:

* STAC is a metadata standard. It links to actual data assets.
* STAC Items typically represent an individual scene: a snapshot at some date and time, covering some space.
* STAC Collections represent a collection of similar items (e.g. scenes) with some shared properties.

## Using STAC

Now that we have the basic concepts out of the way, let's actually *use* STAC to do something interesting. STAC provides enough structure to build APIs to build APIs upon. For example, the Planetary Computer runs an API that lets you search across items to find ones matching some spatio-temporal query. This is just a REST API so you can use any tool that can build HTTP requests. We'll use `pystac-client`.

In [ ]:
import pystac_client

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
catalog

We'll search for Landsat 8 scenes covering some bounding box near St. Louis over part of 2020.

In [ ]:
bbox = [-90.15, 37.85, -89.74, 38.1]

search = catalog.search(
    collections=["landsat-8-c2-l2"],
    bbox=bbox,
    datetime="2020-04-01/2020-11-01"
)

%time items = search.get_all_items()

So in less than a second, we've found narrowed down the millions of Landsat scenes to just ones relevant to our analysis. Let's take a look at one of them.

In [ ]:
item = items[1]
center = shapely.geometry.shape(item.geometry).centroid.bounds[:2][::-1]

m = Map(center=center, zoom=12)
layer = TileLayer(
    url=requests.get(item.assets["tilejson"].href).json()["tiles"][0],
)
m.add_layer(layer)
m.add_control(FullScreenControl())

m.scroll_wheel_zoom = True
m

Perhaps not too surprising: it's cloudy. Remember all those STAC extensions from earlier? The STAC API lets you filter on those. Let's search for scenes that are less that 25% cloudy.

In [ ]:
search = catalog.search(
    collections=["landsat-8-c2-l2"],
    bbox=bbox,
    datetime="2020-04-01/2020-11-01",
    query={"eo:cloud_cover": {"lt": 25}},
)

%time items = search.get_all_items()

In [ ]:
len(items)

## Working with Geospatial data in Python

Now that we've found some items, let's load them up to do our analysis. [xarray](https://xarray.pydata.org/en/stable/) is a natural data container for this type of data. It provides containers for *labeled* n-dimensional arrays with *named* dimensions. In this case, our items can naturally be represented as a 4-d datacube, with dimensions `(time, band, y, x)`.

In [ ]:
import stackstac

epsg = ProjectionExtension.ext(item).epsg
signed_items = [x.to_dict() for x in planetary_computer.sign(items)]
stackstac.stack(signed_items, epsg=epsg)

That's quite the large data array. We don't have time to talk about distributed computing today, so let's trim it down by only loading in

1. A subset of the bands / assets (specifically, the blue, green, red, and near-infrared bands)
2. A subset of the points, by cropping the (large) Landsat 8 scenes down to just our area of interest

In [ ]:
import stackstac

ds = stackstac.stack(
    signed_items, epsg=epsg, assets=["SR_B2", "SR_B3", "SR_B4", "SR_B5"], bounds_latlon=bbox
)
ds

At less than 1 GiB, we can safely load this into memory. We don't really have time to dive into Dask today, but it's being used in the background here.

In [ ]:
from dask.distributed import Client

client = Client()
client

In [ ]:
ds = ds.compute()
ds

xarray supports named dimensions, so let's use those instead of this "SR_B2" stuff.

In [ ]:
common_names = dict(zip(
    ds.band.data.tolist(), ds.common_name.data.tolist()
))
ds = (
    ds.assign_coords(band=[common_names.get(k) for k in ds.band.data])
)
ds

Now we can do all your typical geospatial operations. Band math, like NDVI, uses xarray indexing and normal python operations.

In [ ]:
red = ds.sel(band="red")
nir = ds.sel(band="nir08")
ndvi = (nir - red) / (nir + red)
ndvi

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 12))
ndvi[0].plot.imshow(ax=ax)
ax.set_axis_off();

In [ ]:
fig, ax = plt.subplots()

ndvi.mean(dim=["x", "y"]).plot(ax=ax)
ax.set(title="NDVI over time", ylabel=None);

So that's data access, using STAC.

- We learned the basic concepts of STAC, like Collections to represent Landsat 8 Collection 2 Level-2, and Items to represent individual scenes from that collection.
- We used the Planetary Computer's STAC API to search for items matching some conditions (intersecting our bounding box, in our date range, not too cloudy)
- We used `stackstac` to build an xarray DataArray out of a colleciton of STAC items
- We used xarray to compute NDVI